In [13]:

import time
import torch
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import random
from experiments.pytorch_tablu.utils import preprocess
# import numpy as np
import pandas as pd
# import wandb
from rich import print

from typing import Callable
from datetime import datetime

from time import time
from pytorch_tabular import model_sweep
import warnings

# from pytorch_tabular import TabularModel
# from pytorch_tabular.models import CategoryEmbeddingModelConfig
from pytorch_tabular.config import (
    DataConfig,
    OptimizerConfig,
    TrainerConfig,
    ExperimentConfig,
)
from pytorch_tabular.models.common.heads import LinearHeadConfig


from pytorch_tabular.utils import (
    make_mixed_dataset,
    print_metrics,
    load_covertype_dataset,
)
from pytorch_tabular.utils import get_balanced_sampler, get_class_weighted_cross_entropy

# import os
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:

Feature_Explain = pd.read_excel("data/Amex_ori/Amex Campus.xlsx")
Feature_target_cols = Feature_Explain["Feature Name"][
    Feature_Explain["Extended description"] == "Target"
].to_list()
Feature_cat_col_names = Feature_Explain["Feature Name"][
    Feature_Explain["Feature Type"] == "categorical"
].to_list()
Feature_num_col_names = Feature_Explain["Feature Name"][
    Feature_Explain["Variable Type"] == "numeric"
].to_list()

DEVICE = (
    "mps"
    if torch.backends.mps.is_available()
    else ("gpu" if torch.cuda.is_available() else "cpu")
)
print(DEVICE)

# %%
data = pd.read_parquet("data/Amex_ori/Amex Campus Challenge Train 3.parquet")

data=preprocess(data)

cols = data.columns.to_list()
target_cols = [col for col in cols if col in Feature_target_cols]
if len(target_cols) == 0:
    print("No target column found")
    exit()
# elif len(target_cols) > 1:
#     target_col=target_cols[1]
cat_col_names = [col for col in cols if col in Feature_cat_col_names]
num_col_names = [col for col in cols if col in Feature_num_col_names]

print(f"{target_cols}")
# target_col=cols.unite(Feature_target_cols)

# target_col = 'activation'

# %%

needed_cols = cat_col_names + num_col_names + target_cols
data = data[needed_cols]

data.head()
print(
    f"target_cols={target_cols}, cat_col_names={cat_col_names}, num_col_names={num_col_names}"
)

# %%
# Data: DF, cat_col_names, num_col_names

# 1. activate(0.57%) X recom(12.7%) -> 0~3

# _,data=train_test_split(data,test_size=0.05)




gpu

['ind_recommended', 'activation']

target_cols=['ind_recommended', 'activation'], cat_col_names=['merchant_profile_01'], 
num_col_names=['customer_spend_06', 'customer_spend_07', 'customer_profile_01', 'customer_profile_02', 
'customer_profile_03', 'customer_digital_activity_02', 'customer_profile_04', 'distance_05']

In [16]:
train, test = train_test_split(data,test_size=0.2)
# train, val = train_test_split(train,test_size=0.2)

del data
print(f"train.shape={train.shape}, test.shape={test.shape}")


train.shape=(2784724, 11), test.shape=(309414, 11)

In [17]:
train.describe()


,merchant_profile_01,customer_spend_06,customer_spend_07,customer_profile_01,customer_profile_02,customer_profile_03,customer_digital_activity_02,customer_profile_04,distance_05,ind_recommended,activation
count,2.638385e+06,2.667987e+06,2.667987e+06,2.784362e+06,2.784362e+06,2.767502e+06,2.777636e+06,2.783800e+06,2.784724e+06,2.784724e+06,2.784724e+06
mean,2.603839e+02,2.506789e+02,1.191629e+02,2.941422e+03,2.657957e+03,4.998276e+01,8.476522e+00,1.809109e+02,5.830544e+00,4.999465e-01,5.997004e-03
std,1.388490e+02,3.539141e+02,9.820016e+01,2.128397e+04,1.933483e+04,3.551472e+01,1.850103e+01,1.396270e+02,8.262838e+00,5.000001e-01,7.720778e-02
min,1.010000e+02,1.000000e+00,1.000000e+00,-9.990000e+02,-3.303122e+04,0.000000e+00,0.000000e+00,1.000000e+00,1.060130e-03,0.000000e+00,0.000000e+00
25%,1.010000e+02,4.100000e+01,3.100000e+01,1.296300e+02,1.077200e+02,1.519437e+01,1.166667e+00,6.500000e+01,1.752270e+00,0.000000e+00,0.000000e+00
50%,3.190000e+02,1.420000e+02,9.500000e+01,7.690300e+02,6.230000e+02,4.898091e+01,2.833333e+00,1.570000e+02,3.581526e+00,0.000000e+00,0.000000e+00
75%,4.060000e+02,3.600000e+02,1.950000e+02,2.503830e+03,2.150940e+03,8.532891e+01,7.833333e+00,2.570000e+02,7.402355e+00,1.000000e+00,0.000000e+00
max,5.070000e+02,1.215830e+05,3.600000e+02,4.186172e+06,3.401788e+06,1.000000e+02,2.756500e+03,7.630000e+02,2.540838e+03,1.000000e+00,1.000000e+00


In [18]:

test[['activation', 'ind_recommended']].describe()


,activation,ind_recommended
count,309414.000000,309414.000000
mean,0.006263,0.500482
std,0.078894,0.500001
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,1.000000
75%,0.000000,1.000000
max,1.000000,1.000000


In [19]:

target_col = target_cols[1]

data_config = DataConfig(
    target=[target_col],
    # target should always be a list. Multi-targets are only supported for regression. Multi-Task Classification is not implemented
    continuous_cols=num_col_names,
    categorical_cols=cat_col_names,
    num_workers=0,
)
trainer_config = TrainerConfig(
    auto_lr_find=True,  # Runs the LRFinder to automatically derive a learning rate
    batch_size=1024,
    max_epochs=125,
    early_stopping=None,
    # early_stopping="valid_loss",  # Monitor valid_loss for early stopping
    # early_stopping_mode="min",  # Set the mode as min because for val_loss, lower is better
    # early_stopping_patience=5,  # No. of epochs of degradation training will wait before terminating
    checkpoints="valid_loss",  # Save best checkpoint monitoring val_loss
    checkpoints_path="checkpoints",  # Save the checkpoint in the experiment directory
    # checkpoints_save_top_k=5,
    # progress_bar="simple",
    load_best=True,  # After training, load the best checkpoint
    # accelerator=DEVICE,
    trainer_kwargs=dict(enable_model_summary=False),  # Turning off model summary
)
optimizer_config = OptimizerConfig()

head_config = LinearHeadConfig(
    layers="",  # No additional layer in head, just a mapping layer to output_dim
    dropout=0.1,
    initialization="kaiming",
).__dict__  # Convert to dict to pass to the model config (OmegaConf doesn't accept objects)



In [20]:
from pathlib import Path
def save_sweep_results(sweep_df, save_dir, best_model):
    if not Path(save_dir).exists:
        Path.mkdir(save_dir,parents=True)
    sweep_df.drop(columns=["params", "time_taken", "epochs"])
    sweep_df.to_excel(f"{save_dir}/sweep_result.xlsx", index=False)
    best_model.save_model(f"{save_dir}/best_model.ckpt")


def visualize_sweep_results(sweep_df):
    sweep_df.drop(columns=["params", "time_taken", "epochs"]).style.background_gradient(
        subset=["test_accuracy", "test_f1_score"], cmap="RdYlGn"
    ).background_gradient(subset=["time_taken_per_epoch", "test_loss"], cmap="RdYlGn_r")



In [21]:
%%time
# Filtering out the warnings
# t = time()

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    sweep_df, best_model = model_sweep(
        task="classification",  # One of "classification", "regression"
        train=train,
        test=test,
        data_config=data_config,
        optimizer_config=optimizer_config,
        trainer_config=trainer_config,
        model_list="lite",
        common_model_args=dict(head="LinearHead", head_config=head_config),
        metrics=["accuracy", "f1_score"],
        metrics_params=[{}, {"average": "macro"}],
        metrics_prob_input=[False, True],
        rank_metric=("accuracy", "higher_is_better"),
        progress_bar=True,
        verbose=True,
        suppress_lightning_logger=True,
    )
    



Output()

2024-03-09 00:43:27,787 - {pytorch_tabular:329} - INFO - Training CategoryEmbeddingModel

Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

Failed to compute suggestion for learning rate because there are not enough points. Increase the loop iteration limits or the size of your dataset/dataloader.


2024-03-09 00:46:11,433 - {pytorch_tabular:368} - INFO - Finished Training CategoryEmbeddingModel

2024-03-09 00:46:11,435 - {pytorch_tabular:369} - INFO - Results: model: CategoryEmbeddingModel, # Params: 55 T,   
epochs: 125, test_loss: nan, test_accuracy: 0.9937365651130676, test_f1_score: 0.4984283149242401, time_taken:     
163.67066407203674, time_taken_per_epoch: 1.3093653125762938

2024-03-09 00:46:11,511 - {pytorch_tabular:329} - INFO - Training GANDALFModel

Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

AssertionError: 

In [ ]:

visualize_sweep_results(sweep_df)
save_sweep_results(sweep_df, f"sweep_results for {target_col}",best_model)


NameError: name 'sweep_df' is not defined